In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
from torch.utils.data import DataLoader
import torch.optim as optim

In [ ]:
device='cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device=='cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
training_epochs = 30
learning_rate = 0.001
batch_size = 100

In [ ]:
cirat10_train=dsets.CIFAR10(root='data/cifar10_data/',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)
cirat10_test=dsets.CIFAR10(root='data/cifar10_data/',
                            train=False,
                            transform=transforms.ToTensor(),
                            download=True)

In [ ]:
print(cirat10_train)
print(cirat10_test)

In [ ]:
data_loader=DataLoader(dataset=cirat10_train,
                       batch_size=batch_size,
                       shuffle=True,
                       drop_last=True)

test_loader=DataLoader(dataset=cirat10_test,
                       batch_size=batch_size,
                       shuffle=True,
                       drop_last=True)

In [ ]:
for X, Y in data_loader:
    print(X.size())
    print(Y.size())
    break

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob=0.5

        self.layer1=nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer2=nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer3=nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) (128,4 ,4)
        ) 

        self.fc = nn.Linear(4*4*128, 128, bias=True) #완전연결층
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        #nn.init.uniform_(self.fc.weight) #완전연결층 한정 가중치 초기화

    def forward(self, x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=self.layer3(out)
        out=out.view(out.size(0), -1)
        out=self.fc(out)
        out=self.fc2(out)
        out=self.fc3(out)
        return out    

In [ ]:
model=CNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer=optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
total_batch=len(data_loader)
print(total_batch)

In [ ]:
for epoch in range(training_epochs) :
    avg_cost=0
    
    for X, Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)

        optimizer.zero_grad()
        y_hat = model(X)
        cost = criterion(y_hat, Y)

        cost.backward()
        optimizer.step()
        avg_cost += cost/total_batch
        
    print(epoch, avg_cost)


In [ ]:
accuracys=0
with torch.no_grad():
    for X, Y in test_loader:
        X=X.to(device)
        Y=Y.to(device)
        prediction=model(X)
        correct_prediction = torch.argmax(prediction, 1) == Y
        accuracy=correct_prediction.float().mean()
        print('Accuracy : ', accuracy.item())
        accuracys=accuracy
        print(torch.argmax(prediction,1))
        print(correct_prediction.float().mean().item())
    print('Mean Accuray : ', accuracys)